# Doctor Joins the Scottish Deanery

## Before they can become members of staff they must complete some pre employment checks

These include:

* Identity Verification (We think the GMC Credential + 1 Physical ID Document provides a strong Level of Assurance)
* Right to Work Check
* DBS Check
* GMC Licence

While at the deanery they can complete there compulsory basic training and receive the relevant immunity certificates

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://healthcare-professional-agent:3021 and an api key of MyMedicalAPIKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)


print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [4]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)


def prover_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    
    
    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Object\n")
        print(presentation_request)
        print("\nName : ", presentation_request["name"])
        print("\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n")
        print(presentation_request["requested_attributes"])
        print("\nThere is also a requested_predicates object, which is empty because we did not request any")
        print("\nAnd as with issue credential we have a nonce, which gives the issuer confidence in the liveness of the presentation\n")
        print(presentation_request["nonce"])
    elif state == "presentation_sent":
        
        print("The Presentation object is a bit overwhelming. Let's look at it in detail\n")
        presentation = payload["presentation"]
        
        print("\nThe Requested Proof object contains revealed, unrevealed, self-attested and predicate attributes\n")
        print(presentation["requested_proof"])
        print("\nThe revealed attributes contains the raw and encoded version of the scope attribute. As well as an index into the proof object we will look at later.\n")
        print(presentation["requested_proof"]["revealed_attrs"])
        
        print("\nAll identifiers associated with the presentation")
        print(presentation["identifiers"])
        
        print("\nThen the Proof object - the crypto bit\n")
        proof = presentation["proof"]
        print(proof)
        
        print("\nThis contains an array of proofs, one element for each attribute disclosed (I think)\n")
        print(proof["proofs"])
        
        print("\nEach proof element contains a primary proof and a proof of non-revocation (which will be empty)\n")
        proof_elem = proof["proofs"][0]
        print(proof_elem)
        
        print("\nEach primary proof contains an eq_proof, which is where all the numbers are for the crypto proving an attribute disclosed really is equal to the one signed\n")
        print(proof_elem["primary_proof"]["eq_proof"])
        
        print("\nFinally the proofs are combined into a single aggregated proof object\n")
        print(proof["aggregated_proof"])
        
        print("\nNotice again the c_hash, this is the challenge for the non-interactive ZKP proof which is verifying this presentation\n")
        
    
        
prover_listener = {
    "topic": "present_proof",
    "handler": prover_proof_handler
}

listeners.append(prover_listener)

def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
    


    
holder_listener = {
    "topic": "issue_credential",
    "handler": holder_handler
}

listeners.append(holder_listener)


agent_controller.register_listeners(listeners)

## Connection Made Through Face to Face Meeting (or potentially through website while applying)



In [5]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'df28fb0c-b551-45ff-bf1c-1bc912f308ed', 'recipientKeys': ['7tFfaCBc3TK13be9SZSKPTUbyPvhk83SbxGFkCW76LKV'], 'label': 'Scottish Deanery', 'serviceEndpoint': 'https://d90661b25a39.ngrok.io'}


In [6]:
auto_accept="false"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  d171486b-d3c1-4631-8dec-e58d70000afd
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation


In [7]:
# Label for the connection
my_label = None
# Endpoint you expect to recieve messages at
my_endpoint = None

accept_response = await agent_controller.connections.accept_invitation(connection_id, my_label, my_endpoint)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  d171486b-d3c1-4631-8dec-e58d70000afd
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  d171486b-d3c1-4631-8dec-e58d70000afd
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  d171486b-d3c1-4631-8dec-e58d70000afd
State :  active
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
Connection ID: d171486b-d3c1-4631-8dec-e58d70000afd is now active.

---------------------------------------------------------------------

Handle present-proof
Connection ID :  d1714

## Is asked to Present their GMC Credential

If online this might simply be proof of non-revoked GMC Number attribute. During a face to face interaction, verifiers may wish to see the photograph within the licence for additional assurance.

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [11]:
# Optional Query parameters
verifier_connection_id = connection_id
thread_id=None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)

# We fetch the first record from the response. You may want to customise this further
presentation_record = proof_records_response["results"][0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [12]:
# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials("a9535817-9a11-46e6-86f8-c1aeae6f620f")
print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")
# print(credentials)

credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}


# Note we are working on a friendlier api to abstract this away

if credentials:
    for credential in credentials:

        for attribute_name in credential["presentation_referents"]:
            if attribute_name not in credentials_by_reft:
                credentials_by_reft[attribute_name] = credential

for (key, value) in credentials_by_reft.items():
    print(f"Attribute {key} can be satisfied by Credential with Referent {value['cred_info']['referent']}")
                
for attribute_name in presentation_record["presentation_request"]["requested_attributes"]:
    if attribute_name in credentials_by_reft:
        revealed[attribute_name] = {
            "cred_id": credentials_by_reft[attribute_name]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }


print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}

Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal


Generate the proof


## Send Presentation of GMC Licence Attributes

A presentation is sent in represent to a presentation record that has previously been created.

In [ ]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)

# Complete Right to Work Check

## Receive Evidence of Successful Right to Work Check

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
issuer_connection_id = connection_id
thread_id=None
state = "offer_received"
role = "prover"


records_response = await agent_controller.issuer.get_records(state=state)
rtw_record = records_response["results"][0]
rtw_record_id = rtw_record["credential_exchange_id"]

In [ ]:

for attribute in rtw_record['credential_proposal_dict']['credential_proposal']['attributes']:

    print("Arribute Offered")
    print(f"{attribute['name']}:{attribute['value']}")

In [ ]:
rtw_response = await agent_controller.issuer.send_request_for_record(rtw_record_id)

In [ ]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "RTW"

store_cred_response = await agent_controller.issuer.store_credential(rtw_record_id, credential_id)

# Complete DBS Check

Likely this is done async by the Scottish Deanery and could be issued on successful completion.

## Receive Evidence of Successful DBS Check

In [ ]:
issuer_connection_id = connection_id
thread_id=None
state = "offer_received"
role = "prover"


records_response = await agent_controller.issuer.get_records(state=state)
dbs_record = records_response["results"][0]
dbs_record_id = dbs_record["credential_exchange_id"]

In [ ]:

for attribute in dbs_record['credential_proposal_dict']['credential_proposal']['attributes']:

    print("Arribute Offered")
    print(f"{attribute['name']}:{attribute['value']}")

In [ ]:
dbs_response = await agent_controller.issuer.send_request_for_record(dbs_record_id)

In [ ]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "DBS Check"

store_cred_response = await agent_controller.issuer.store_credential(dbs_record_id, credential_id)

# Receive Necessary Immunisations

These may arranged through the Deanery, but would likely be done by a different entity. Perhaps the Deanery is just responsible for providing digital credential attesting to the immunisations

## Receive Immunity Certificate

Note the data in the certificate is currently held within a single JSON attribute

In [ ]:
issuer_connection_id = connection_id
thread_id=None
state = "offer_received"
role = "prover"


records_response = await agent_controller.issuer.get_records(state=state)
ic_record = records_response["results"][0]
ic_record_id = ic_record["credential_exchange_id"]

In [ ]:

for attribute in ic_record['credential_proposal_dict']['credential_proposal']['attributes']:

    print("Arribute Offered")
    print(f"{attribute['name']}:{attribute['value']}")

In [ ]:
dbs_response = await agent_controller.issuer.send_request_for_record(ic_record_id)

In [ ]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "Immunity Certificate"

store_cred_response = await agent_controller.issuer.store_credential(ic_record_id, credential_id)

# Complete Compulsory Basic Training

Typical onboarding into the NHS, often repetitive. This would provide generic training to all healthcare professionals, that they would be able to prove they have completed so that individual organisations only need to cover specific nuances of their assignment

Whether the Deanery is the ideal issuer of this information would need to be figured out within the organisations.

Note, perhaps this is as simple as completing an online set of videos

## Receive CBT Certificate

Note the data in the certificate is currently held within a single JSON attribute

In [ ]:
issuer_connection_id = connection_id
thread_id=None
state = "offer_received"
role = "prover"


records_response = await agent_controller.issuer.get_records(state=state)
cbt_record = records_response["results"][0]
cbt_record_id = cbt_record["credential_exchange_id"]

In [ ]:

for attribute in cbt_record['credential_proposal_dict']['credential_proposal']['attributes']:

    print("Arribute Offered")
    print(f"{attribute['name']}:{attribute['value']}")

In [ ]:
dbs_response = await agent_controller.issuer.send_request_for_record(cbt_record_id)

In [ ]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "CBT Certificate"

store_cred_response = await agent_controller.issuer.store_credential(cbt_record_id, credential_id)

## Let's Look at the Credentials this Junior Doctor has in their Wallet now

In [ ]:
response = await agent_controller.credentials.get_all()
for credential in response["results"]:
    print("-"*40)
    print(f"Credential - {credential['referent']} \n ")
    print("-"*40)
    print("Attributes \n")
    for (key, val) in credential["attrs"].items():
        print(f"Attribute Name : {key}")
        if key == "Base64Image":
            print("Value : Some large base64 image string")
        else:
            print(f"Value : {val} \n")
        

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()